In [764]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [765]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в значениях
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if 'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Все в порядке' if len(temp_set) == 1 else 'Ошибка!!!'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом
    
    try:
        return float(x)
    except ValueError:
        return 0
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0
    
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

def clean_column(df,column):
    """
    Функция для очистки от колонки от пустых значений
    """
    df[column] = df[column].astype(str) # Приводим к строковому виду
    df[column] = df[column].apply(lambda x:x.replace('nan','Отсутствует'))
    df[column] = df[column].apply(lambda x:x.strip()) # очищаем от пробельных символов(на случай если в ячейке стоит просто пробел или несколько пробелов)
    df[column] = df[column].apply(lambda x:x if x else 'Отсутствует') # Если пустая строка то заменяем на значение Не заполнено
    

In [766]:
group_rb_region_transfer_3_to_reestr = 0

file_transfer_to_upp = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'

# file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
# file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Заголовок Таблица 3 от 08.11.xlsx'

In [767]:
# Создаем список колонок которые нужно загрузить
use_cols_upp = list(range(6))
use_cols_table3 = list(range(34))

# Загружаем датафреймы
df = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols_upp)# Реестр УПП
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=use_cols_table3) # Таблица 3

In [768]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

In [769]:
"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
"""

'\nСоответствие названий колонок используемым в программе номерам колонок\nЛесничество -1\nУчастковое лесничество- 2\nУрочище - 3\nНомер лесного квартала -4\nНомер лесотаксационного выдела -5\nПлощадь лесотаксационного выдела, га - 6\n'

In [770]:
df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га'},inplace=True)



In [771]:

clean_column(df,'Урочище') # очищаем от пустых строк и пробелов

In [772]:
df.head(15)

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га"
0,Ангоянское,Ангоянское,Не заполнено,32,1,13
1,Ангоянское,Ангоянское,Не заполнено,32,2,7
2,Ангоянское,Ангоянское,Не заполнено,32,3,11
3,Ангоянское,Ангоянское,Не заполнено,32,5,10
4,Ангоянское,Ангоянское,Не заполнено,32,7,78
5,Ангоянское,Ангоянское,Не заполнено,32,7,78
6,Ангоянское,Ангоянское,Не заполнено,32,7,78
7,Ангоянское,Ангоянское,Не заполнено,32,8,5
8,Ангоянское,Ангоянское,Не заполнено,32,9,12
9,Ангоянское,Ангоянское,Не заполнено,32,9,12


In [773]:
df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
    lambda x: x.replace(',', '.'))

# Группируем
df_upp = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                       'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                               })

df_upp=df_upp.astype(str)


In [774]:
# Извлекаем индексы в колонки
df_upp = df_upp.reset_index()

In [775]:
df_upp.head()


,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га"
0,Ангоянское,Ангоянское,Не заполнено,32,1,13.0
1,Ангоянское,Ангоянское,Не заполнено,32,2,7.0
2,Ангоянское,Ангоянское,Не заполнено,32,3,11.0
3,Ангоянское,Ангоянское,Не заполнено,32,5,10.0
4,Ангоянское,Ангоянское,Не заполнено,32,7,78.0


In [776]:
# Обрабатываем колонки таблицы 3
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [777]:
# делаем строковыми первые 5 колонок в датафрейме Реестр УПП
df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']] = df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']].astype(str)
df_upp['ID'] = df_upp.loc[:,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела'] ].sum(axis=1)
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace(' ',''))
df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace('nan',''))

In [778]:
# Очищаем от нан и пустых строк
clean_column(df_table_3,'7')

In [779]:
# делаем строковыми колонки по которым будем соединять в датафрейме 3 таблицы
df_table_3[['5','6','7','8','9']] = df_table_3[['5','6','7','8','9']].astype(str)
df_table_3['ID'] = df_table_3.loc[:,['5','6','7','8','9'] ].sum(axis=1)
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace(' ',''))
df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace('nan',''))

In [780]:
# Мержим по полю айди
merged_df = pd.merge(df_upp,df_table_3,how='outer',left_on='ID',right_on='ID',indicator=True)

In [781]:
merged_df.head()

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",ID,1,2,3,...,26,27,28,29,30,31,32,33,34,_merge
0,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.2,...,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2015.0,both
1,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.2,...,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2020.0,both
2,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.3,...,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2021.0,both
3,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,2.3,...,более 40%,2022-04-05 00:00:00,1.0,0.0,1.0,103.0,NaN,2.3,2015.0,both
4,Ангоянское,Ангоянское,Не заполнено,32,2,7.0,АнгоянскоеАнгоянскоеНезаполнено322,NaT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [782]:
merged_df.to_excel('Тест.xlsx',index=False)

In [783]:
merged_df['Площадь'] = np.where(merged_df['_merge'] == 'both',merged_df['Площадь лесотаксационного выдела, га'],merged_df['10'])

In [784]:
# Приводим к инту колонки чтобы отсортировать их
merged_df['8'] = merged_df['8'].apply(convert_to_int_transfer)
merged_df['9'] = merged_df['9'].apply(convert_to_int_transfer)

In [785]:
merged_df.sort_values(['5','6','7','8','9'],inplace=True)

# Присваиваем колонке 10 значение колонки созданной колонки 10
#merged_df['10'] = merged_df['Площадь']

In [786]:
merged_df.head()

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",ID,1,2,3,...,27,28,29,30,31,32,33,34,_merge,Площадь
0,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.2,...,0,0.0,0.0,1.0,201.0,NaN,0.0,2015.0,both,13.0
1,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.2,...,0,0.0,0.0,1.0,201.0,NaN,0.0,2020.0,both,13.0
2,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.3,...,0,0.0,0.0,1.0,201.0,NaN,0.0,2021.0,both,13.0
3,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,2.3,...,2022-04-05 00:00:00,1.0,0.0,1.0,103.0,NaN,2.3,2015.0,both,13.0
1963,Бабушкинское,Бабушкинское,Не заполнено,91,2,6.4,БабушкинскоеБабушкинскоеНезаполнено912,NaT,NaN,0.6,...,2022-08-31 00:00:00,1.0,0.0,1.0,102.0,NaN,0.6,2018.0,both,6.4


In [787]:
# Создаем документ
wb = openpyxl.Workbook()
# создаем листы
ren_sheet = wb['Sheet']
ren_sheet.title = 'Нет в таблице 3'
wb.create_sheet(title='Нет в реестре УПП', index=1)
wb.create_sheet(title='Выделы с обновленной площадью', index=2)

<Worksheet "Выделы с обновленной площадью">

In [788]:
# Записываем каждый датафрейм в соответсвующий лист
left_df = merged_df[merged_df['_merge'] == 'left_only']
left_df.drop(left_df.iloc[:,5:],axis=1,inplace=True)

In [789]:
# Очищаем от пробелов и строки названия урочища не заполнено
left_df['Урочище'] = left_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
left_df['Урочище'] = left_df['Урочище'].apply(lambda x:x.strip())

In [790]:
for r in dataframe_to_rows(left_df, index=False, header=True):
    wb['Нет в таблице 3'].append(r)
wb['Нет в таблице 3'].column_dimensions['A'].width = 30
wb['Нет в таблице 3'].column_dimensions['B'].width = 30
    

In [791]:
# Записываем каждый датафрейм в соответсвующий лист
right_df = merged_df[merged_df['_merge'] == 'right_only']
right_df.drop(right_df.iloc[:,:6],axis=1,inplace=True)

In [792]:
# Удаляем лишние колонки
right_df.drop(right_df.iloc[:,10:],axis=1,inplace=True)

right_df.drop(columns=['ID','1','2','3','4'],axis=1,inplace=True)

In [793]:
right_df

,5,6,7,8,9
139555,Бабушкинское,Бабушкинское,Не заполнено,238,12
139556,Бабушкинское,Бабушкинское,Не заполнено,238,12
139557,Бабушкинское,Бабушкинское,Не заполнено,253,17
139558,Бабушкинское,Бабушкинское,Не заполнено,263,3
139559,Бабушкинское,Бабушкинское,Не заполнено,264,7
...,...,...,...,...,...
139550,Хандагатайское,Челутаевское,Не заполнено,161,14
139551,Хандагатайское,Челутаевское,Не заполнено,161,17
139552,Хандагатайское,Челутаевское,Не заполнено,173,11
139553,Хандагатайское,Челутаевское,Не заполнено,173,15


In [794]:
# Очищаем от пробелов и строки названия урочища не заполнено
right_df['7'] = right_df['7'].apply(lambda x:x.replace('Название урочища не заполнено',''))
right_df['7'] = right_df['7'].apply(lambda x:x.strip())

In [795]:
# Переименовываем колонки
right_df.columns = ['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']

In [796]:
# Записываем на лист
for r in dataframe_to_rows(right_df, index=False, header=True):
    wb['Нет в реестре УПП'].append(r)

wb['Нет в реестре УПП'].column_dimensions['A'].width = 30
wb['Нет в реестре УПП'].column_dimensions['B'].width = 30

In [797]:
# Записываем каждый датафрейм в соответсвующий лист
both_df = merged_df[merged_df['_merge'] == 'both']

In [798]:
both_df.drop(both_df.iloc[:,17:-1],axis=1,inplace=True)

In [799]:
both_df.head(5)

,Лесничество,Участковое лесничество,Урочище,Номер лесного квартала,Номер лесотаксационного выдела,"Площадь лесотаксационного выдела, га",ID,1,2,3,4,5,6,7,8,9,10,Площадь
0,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,13.0
1,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,300.0,13.0
2,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,500.0,13.0
3,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,АнгоянскоеАнгоянскоеНезаполнено321,2022-06-29,NaN,2.3,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,400.0,13.0
1963,Бабушкинское,Бабушкинское,Не заполнено,91,2,6.4,БабушкинскоеБабушкинскоеНезаполнено912,NaT,NaN,0.6,Республика Бурятия,Бабушкинское,Бабушкинское,Не заполнено,91,2,7777.0,6.4


In [800]:
both_df.drop(columns=['ID','1','2','3','4','5','6','7','8','9','Площадь лесотаксационного выдела, га'],axis=1,inplace=True)

In [801]:
# Очищаем от пробелов и строки названия урочища не заполнено
both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.strip())

In [802]:
both_df.rename(columns={'Площадь':'Площадь из реестра УПП','10':'Старая площадь'},inplace=True)

In [803]:
for r in dataframe_to_rows(both_df, index=False, header=True):
    wb['Выделы с обновленной площадью'].append(r)

wb['Выделы с обновленной площадью'].column_dimensions['A'].width = 30
wb['Выделы с обновленной площадью'].column_dimensions['B'].width = 30

In [804]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
wb.save(f'{path_to_end_folder}/Данные для проверки корректности обновления площадей в таблице 3 от {current_time}.xlsx')

In [805]:
# Делаем таблицу 3
out_table3_df = merged_df[merged_df['_merge'] != 'left_only']

In [806]:
# Приводим к инту колонки чтобы отсортировать их
out_table3_df['8'] = out_table3_df['8'].apply(convert_to_int_transfer)
out_table3_df['9'] = out_table3_df['9'].apply(convert_to_int_transfer)

In [807]:
out_table3_df.sort_values(['5','6','7','8','9'],inplace=True)

In [808]:
# Присваиваем колонке 10 значение колонки созданной колонки 10
out_table3_df['10'] = out_table3_df['Площадь']

In [809]:
out_table3_df.drop(columns=['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Площадь лесотаксационного выдела, га','ID','_merge','Площадь'],inplace=True)

In [810]:
# Очищаем от пробелов и строки названия урочища не заполнено
out_table3_df['7'] = out_table3_df['7'].apply(lambda x:x.replace('Название урочища не заполнено',''))
out_table3_df['7'] = out_table3_df['7'].apply(lambda x:x.strip())


In [811]:
out_table3_df.head(3)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,...,0,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2015.0
1,2022-06-29,NaN,0.2,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,...,0,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2020.0
2,2022-06-29,NaN,0.3,Республика Бурятия,Ангоянское,Ангоянское,Не заполнено,32,1,13.0,...,0,0,0,0.0,0.0,1.0,201.0,NaN,0.0,2021.0


In [812]:
# Приводим даты к нормальному виду ДД.ММ.ГГГГ
out_table3_df['1'] = pd.to_datetime(out_table3_df['1'],errors='coerce',dayfirst=True)
out_table3_df['22'] = pd.to_datetime(out_table3_df['22'],errors='coerce',dayfirst=True)
out_table3_df['27'] = pd.to_datetime(out_table3_df['27'],errors='coerce',dayfirst=True)
out_table3_df['1'] = out_table3_df['1'].apply(create_doc_convert_date)
out_table3_df['22'] = out_table3_df['22'].apply(create_doc_convert_date)
out_table3_df['27'] = out_table3_df['27'].apply(create_doc_convert_date)

In [813]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
itog = load_workbook(header_table3) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(out_table3_df,index=False,header=False):
    itog['Лист1'].append(row)

In [814]:
itog.save(f'{path_to_end_folder}/Таблица 3 с присвоенными площадями из реестра УПП от {current_time}.xlsx')